If you do not have sentinelsat package installed on your machine run the command below

In [ ]:
import sys
!{sys.executable} -m pip install sentinelsat

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import os
import geopandas as gpd

import zipfile
import rasterio 
from rasterio.mask import mask

from shapely.geometry import MultiPolygon, Polygon

In [2]:
os.chdir('..')

In [3]:
os.getcwd()

'/Users/ksunil/Desktop/Karan/Codebase'

In [31]:
def get_sentinel_images(data_dirs, reef_name, start_date, end_date, num_images):
    #login into api
    user = 'karans04'
    password = 'igpp2019'
    api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
    
    #load in geojson of reef
    reef_gjson_fp = os.path.join(data_dirs, reef_name ,reef_name+'.geojson')
    reef_footprint = geojson_to_wkt(read_geojson(reef_gjson_fp))
    
    #query sentinel sat api
    products = api.query(reef_footprint,date = (start_date, end_date),platformname = 'Sentinel-2'\
                         ,area_relation = 'Intersects',processinglevel = 'Level-2A',\
                         cloudcoverpercentage = (0,15),order_by = 'cloudcoverpercentage')
    sorted_gdf = api.to_geodataframe(products).sort_values('cloudcoverpercentage')
#     display(sorted_gdf.T)
#     return
    #creating folder for saving sentinel images
    sentinel_path = os.path.join(data_dirs,reef_name, 'SAFE files')
    if not os.path.exists(sentinel_path):
        os.makedirs(sentinel_path)

    #downloading num_images    
    for i,x in enumerate(sorted_gdf.index):
        if i < num_images:
            fn = sorted_gdf.iloc[i].title
            fp = os.path.join(sentinel_path, fn)
            if not os.path.exists(fp):
                api.download(x, directory_path = sentinel_path)
    #unzipping files
    for file in os.listdir(sentinel_path):
        file_path = os.path.join(sentinel_path, file)
        out_path = os.path.join(sentinel_path, file.split('.')[0])
        if not os.path.exists(out_path):
            with zipfile.ZipFile(file_path,"r") as zip_ref:
                zip_ref.extractall(out_path)
                os.remove(file_path)


In [32]:
get_sentinel_images('data', 'Nasau', '20160101', '20200520', 20)

Downloading: 100%|██████████| 916M/916M [26:05<00:00, 509kB/s] 
MD5 checksumming: 100%|██████████| 916M/916M [00:01<00:00, 555MB/s]
Downloading: 100%|██████████| 674M/674M [10:11<00:00, 636kB/s]
MD5 checksumming: 100%|██████████| 674M/674M [00:01<00:00, 556MB/s]
Downloading: 100%|██████████| 697M/697M [12:14<00:00, 949kB/s] 
MD5 checksumming: 100%|██████████| 697M/697M [00:01<00:00, 555MB/s]
Downloading: 100%|██████████| 983M/983M [19:23<00:00, 801kB/s]
MD5 checksumming: 100%|██████████| 983M/983M [00:01<00:00, 555MB/s]
Product 6481e7bf-3937-4a3d-9075-4a0c26e79c6c is not online. Triggering retrieval from long term archive.
Downloading: 100%|██████████| 741M/741M [16:57<00:00, 1.12MB/s]
MD5 checksumming: 100%|██████████| 741M/741M [00:01<00:00, 556MB/s]
Downloading: 100%|██████████| 517M/517M [08:53<00:00, 506kB/s]
MD5 checksumming: 100%|██████████| 517M/517M [00:00<00:00, 518MB/s]
Downloading: 100%|██████████| 525M/525M [13:47<00:00, 635kB/s]
MD5 checksumming: 100%|██████████| 525M/525